In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [4]:
train_dataset = torchvision.datasets.MNIST(root='data',
                                          train=True,
                                          download=True,
                                          transform=ToTensor())

test_dataset = torchvision.datasets.MNIST(root='data',
                                         train=False,
                                         download=True,
                                         transform=ToTensor())

In [5]:
train_dataloader = DataLoader(train_dataset,
                             batch_size=batch_size,
                             shuffle=True)

test_dataloader = DataLoader(test_dataset,
                            batch_size=batch_size,
                            shuffle=False)

In [6]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [7]:
model = ConvNet(num_classes).to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
total_step = len(train_dataloader)
for epoch in range(num_epochs):
    for i, (img, label) in enumerate(train_dataloader):
        img = img.to(device)
        label = label.to(device)
        
        output = model(img)
        loss = criterion(output, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch: [{}/{}], Step: [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss))

Epoch: [1/5], Step: [100/600], Loss: 0.0870
Epoch: [1/5], Step: [200/600], Loss: 0.1001
Epoch: [1/5], Step: [300/600], Loss: 0.1088
Epoch: [1/5], Step: [400/600], Loss: 0.0853
Epoch: [1/5], Step: [500/600], Loss: 0.0194
Epoch: [1/5], Step: [600/600], Loss: 0.0960
Epoch: [2/5], Step: [100/600], Loss: 0.0367
Epoch: [2/5], Step: [200/600], Loss: 0.0481
Epoch: [2/5], Step: [300/600], Loss: 0.0593
Epoch: [2/5], Step: [400/600], Loss: 0.0301
Epoch: [2/5], Step: [500/600], Loss: 0.0232
Epoch: [2/5], Step: [600/600], Loss: 0.0263
Epoch: [3/5], Step: [100/600], Loss: 0.0232
Epoch: [3/5], Step: [200/600], Loss: 0.0609
Epoch: [3/5], Step: [300/600], Loss: 0.0353
Epoch: [3/5], Step: [400/600], Loss: 0.0274
Epoch: [3/5], Step: [500/600], Loss: 0.0308
Epoch: [3/5], Step: [600/600], Loss: 0.0426
Epoch: [4/5], Step: [100/600], Loss: 0.0053
Epoch: [4/5], Step: [200/600], Loss: 0.0159
Epoch: [4/5], Step: [300/600], Loss: 0.0233
Epoch: [4/5], Step: [400/600], Loss: 0.0018
Epoch: [4/5], Step: [500/600], L

In [10]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for img, label in test_dataloader:
        img = img.to(device)
        label = label.to(device)
        output = model(img)
        _, predicted = torch.max(output.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
        
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98.86 %
